In [2]:
import os, requests, bs4, re, collections
import lxml
import warnings; warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
import pandas as pd
#import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import argparse
import cv2

from PIL import Image
import requests
from io import BytesIO

from PIL import Image, ImageDraw
import argparse
import sys

In [3]:
table_movie = pd.read_csv("clean_data.csv")

In [4]:
table_movie.head()

,Titre,Année,Durée,Genre,image
0,Sans répit,2022,95,"\nAction, Crime, Thriller",https://m.media-amazon.com/images/M/MV5BZjQyMT...
1,Benedetta,2021,131,"\nBiography, Drama, History",https://m.media-amazon.com/images/M/MV5BNGRiMm...
2,Titane,2021,108,"\nDrama, Horror, Sci-Fi",https://m.media-amazon.com/images/M/MV5BNGJlMT...
3,Les exploits d'un jeune don juan,1986,95,\nComedy,https://m.media-amazon.com/images/M/MV5BNjJiYW...
4,La Vie d'Adèle,2013,180,"\nDrama, Romance",https://m.media-amazon.com/images/M/MV5BMTQ5NT...


In [ ]:
images_des_fims = table_movie["image"].values.tolist()
year = table_movie["Année"].values.tolist()
color_image = []
year_movie = []
nb_error = 0
numcolors=2
resize=150
for x in range(len(images_des_fims)):
    try:
        response = requests.get(images_des_fims[x])
        img = Image.open(BytesIO(response.content))
        img = img.copy()
        img.thumbnail((resize, resize))
        paletted = img.convert('P', palette=Image.ADAPTIVE, colors=numcolors)

        palette = img.getpalette()
        color_counts = sorted(paletted.getcolors(), reverse=True)

        for i in range(2):
            palette_index = color_counts[i][1]
            dominant_color = palette[palette_index*3:palette_index*3+3]
            color_image.append(tuple(dominant_color))
            year_movie.append(year[x])
    except:
        nb_error += 1
        

In [69]:
df = table_movie[table_movie["image"] != "https://m.media-amazon.com/images/M/MV5BMTk3MDIyMTcwNV5BMl5BanBnXkFtZTcwMDU0MDYyMQ@@._V1_UX67_CR0,0,67,98_AL_.jpg"]

In [73]:
titre = df["Titre"].values.tolist()
durée = df["Durée"].values.tolist()
genre = df["Genre"].values.tolist()
titre_image = []
durée_image = []
genre_image = []
for x in range(len(titre)):
    for i in range(2):
        titre_image.append(titre[x])
        durée_image.append(durée[x])
        genre_image.append(genre[x])
        

In [74]:
len(titre_image)

19998

In [76]:
table_color = pd.DataFrame(columns = ["Titre","Année", "Durée","Genre", "Color"])
table_color["Titre"] = titre_image
table_color["Année"]=  year_movie
table_color["Durée"]= durée_image
table_color["Genre"]= genre_image
table_color["Color"] = color_image

In [5]:
couleur = table_color["Color"]
couleur[1] = couleur[1].replace(",", " ")
couleur[1] = couleur[1].replace(")", "")
couleur[1] = couleur[1].replace("(", "")
split = couleur[1].split()
rgb = ( int(split[0]), int(split[1]), int(split[2]))

In [6]:
from scipy.spatial import KDTree
from webcolors import (
    CSS3_HEX_TO_NAMES,
    hex_to_rgb,
)

In [48]:
couleur = table_color["Color"]
color_movies = []
for i in range(len(couleur)):
    couleur[i] = couleur[i].replace(",", " ")
    couleur[i] = couleur[i].replace(")", "")
    couleur[i] = couleur[i].replace("(", "")
    split = couleur[i].split()
    rgb = ( int(split[0]), int(split[1]), int(split[2]))
    css3_db = CSS3_HEX_TO_NAMES
    names = []
    rgb_values = []
    
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(hex_to_rgb(color_hex))

    kdt_db = KDTree(rgb_values)
    distance, index = kdt_db.query(rgb)
    color_movies.append(names[index])

In [50]:
table_color["Color_Name"] = color_movies

In [13]:
clean = table_color["Année"].values.tolist()
for i in range(len(clean)):
    try:
        clean[i] = int(clean[i])
    except:
        clean[i] = clean[i].replace("I", " ")
        clean[i] = clean[i].replace("V", " ")
        try: 
            clean[i] = int(clean[i])
        except:
            clean[i] = 0
            
table_color["Année"] = clean           

In [14]:
table_color = table_color[table_color["Année"] != 0]

In [15]:
table_color.Color_Name.unique()

array(['darkslategray', 'gray', 'black', 'dimgray', 'purple', 'sienna',
       'lightslategray', 'gainsboro', 'saddlebrown', 'maroon', 'white',
       'darkgray', 'rosybrown', 'darkolivegreen', 'antiquewhite', 'brown',
       'darksalmon', 'silver', 'darkseagreen', 'whitesmoke',
       'palegoldenrod', 'darkkhaki', 'cadetblue', 'dodgerblue', 'snow',
       'tan', 'wheat', 'linen', 'midnightblue', 'sandybrown',
       'darkgoldenrod', 'darkslateblue', 'mediumaquamarine', 'burlywood',
       'powderblue', 'slategray', 'lavender', 'floralwhite', 'darkblue',
       'steelblue', 'lightgray', 'indianred', 'khaki', 'peru',
       'mediumturquoise', 'teal', 'bisque', 'lightblue', 'seashell',
       'skyblue', 'seagreen', 'ghostwhite', 'darkred', 'goldenrod',
       'firebrick', 'beige', 'olive', 'gold', 'blanchedalmond', 'red',
       'crimson', 'mediumvioletred', 'lightcoral', 'mintcream',
       'lightsteelblue', 'darkcyan', 'oldlace', 'olivedrab', 'darkgreen',
       'thistle', 'cornflowerb

In [70]:
table_color.head()

,Titre,Année,Durée,Genre,Color,Color_Name,Main Color
0,Sans répit,2022,95,"\nAction, Crime, Thriller",55 47 32,darkslategray,Main Color
1,Sans répit,2022,95,"\nAction, Crime, Thriller",152 147 132,gray,Secondary Color
2,Benedetta,2021,131,"\nBiography, Drama, History",6 4 3,black,Main Color
3,Benedetta,2021,131,"\nBiography, Drama, History",110 89 76,dimgray,Secondary Color
4,Titane,2021,108,"\nDrama, Horror, Sci-Fi",152 27 91,purple,Main Color


In [84]:
col = table_color.groupby("Color_Name")["Durée"].count().reset_index(name='Count').sort_values(['Count'], ascending=False)

In [68]:
col.head(20)

,Color_Name,Count
6,black,2455
30,darkslategray,2251
32,dimgray,1762
40,gray,1238
19,darkgray,1184
23,darkolivegreen,898
93,rosybrown,793
101,silver,772
100,sienna,631
95,saddlebrown,577


In [18]:
type_color = []
a = "Main Color"
b = "Secondary Color"
for x in range(len(table_color)):
    if x%2 == 0:
        type_color.append(a)
    else:
        type_color.append(b)       

In [19]:
table_color["Main Color"] = type_color

In [20]:
len(table_color)

19652

In [20]:
table_color.head(6
                )

,Titre,Année,Durée,Genre,Color,Color_Name,Main Color,Base_Color
0,Sans répit,2022,95,"\nAction, Crime, Thriller",55 47 32,darkslategray,Main Color,cyan
1,Sans répit,2022,95,"\nAction, Crime, Thriller",152 147 132,gray,Secondary Color,black
2,Benedetta,2021,131,"\nBiography, Drama, History",6 4 3,black,Main Color,black
3,Benedetta,2021,131,"\nBiography, Drama, History",110 89 76,dimgray,Secondary Color,gray
4,Titane,2021,108,"\nDrama, Horror, Sci-Fi",152 27 91,purple,Main Color,purple
5,Titane,2021,108,"\nDrama, Horror, Sci-Fi",149 86 93,sienna,Secondary Color,brown


In [22]:
main_color = table_color[table_color["Main Color"] == "Main Color"]
second_color = table_color[table_color["Main Color"] != "Main Color"]

print(len(main_color))
print(len(second_color))

9826
9826
